# OS 확인

In [0]:
!cat /etc/issue.net

Ubuntu 18.04.2 LTS


# 하드웨어 사양

In [0]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power ma

# Memory

In [0]:
!cat /proc/meminfo

MemTotal:       13335276 kB
MemFree:        11166932 kB
MemAvailable:   12649152 kB
Buffers:           81636 kB
Cached:          1556140 kB
SwapCached:            0 kB
Active:           551212 kB
Inactive:        1364648 kB
Active(anon):     248336 kB
Inactive(anon):      352 kB
Active(file):     302876 kB
Inactive(file):  1364296 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               904 kB
Writeback:             0 kB
AnonPages:        278148 kB
Mapped:           169380 kB
Shmem:               904 kB
Slab:             144040 kB
SReclaimable:     112364 kB
SUnreclaim:        31676 kB
KernelStack:        3680 kB
PageTables:         4632 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667636 kB
Committed_AS:    1733252 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

# Disk

In [0]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         359G   25G  316G   8% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G   12K  6.4G   1% /var/colab
/dev/sda1       365G   29G  336G   8% /opt/bin
shm             6.0G     0  6.0G   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware


# GPU

In [0]:
!nvidia-smi

Tue May 21 06:11:29 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# 구글 드라이브와 Colab연동

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!cd 'gdrive/My Drive/data/snli_1.0'; ls -al;

total 876986
-rw------- 1 root root      5828 May 20 08:38 README.txt
-rw------- 1 root root   9745714 May 20 08:39 snli_1.0_dev.jsonl
-rw------- 1 root root   7565773 May 20 08:39 snli_1.0_dev.txt
-rw------- 1 root root   9730457 May 20 08:39 snli_1.0_test.jsonl
-rw------- 1 root root   7550390 May 20 08:39 snli_1.0_test.txt
-rw------- 1 root root 487457790 May 20 08:40 snli_1.0_train.jsonl
-rw------- 1 root root 375697923 May 20 08:39 snli_1.0_train.txt
-rw------- 1 root root    277673 May 20 08:38 vocab.txt


# Import

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

from tqdm import tqdm
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

# Define training function

In [0]:
regxs = list("[@_!#$%^&*()<>?/\|}{~:]'\".,-;`+=")

def strip_string(string):
    for regx in regxs:
        string = string.replace(regx, " " + regx + " ")
    return string.lower().strip()

  
# label index 정의
label_dict = { "neutral": 0, "entailment": 1, "contradiction": 2 }


"""
pickle로 vocab 로드
"""
def load_vocab(file="Data/vocab.txt"):
    vocab = {}
    index = 0
    with open(file, "r") as f:
        for token in f:
            vocab[token.strip()] = index
            index += 1
    return vocab

  
"""
파일로 부터 vocab을 이용해 데이터 로드
"""
def load_data(file, vocab):
    gold_label = []
    sentence1 = []
    sentence2 = []

    dataset = pd.read_csv(file, sep="\t")
    for i, row in dataset.iterrows():
        if row['gold_label'] == "-" or pd.isnull(row['sentence1']) or pd.isnull(row['sentence2']):
            continue

        gold_label.append(label_dict[row['gold_label']])

        line1 = []
        for token in strip_string(row['sentence1']).split():
            line1.append(vocab[token])
        while len(line1) < 82:
            line1.append(1) # <pad>
        sentence1.append(line1)

        line2 = []
        for token in strip_string(row['sentence2']).split():
            line2.append(vocab[token])
        while len(line2) < 63:
            line2.append(1) # <pad>
        sentence2.append(line2)
    
    return gold_label, sentence1, sentence2


def build_tensor(label, sentence1, sentence2, batch_size=256):
    torch_labe = torch.tensor(label, dtype=torch.long)
    torch_sentence1 = torch.tensor(sentence1, dtype=torch.long)
    torch_sentence2 = torch.tensor(sentence2, dtype=torch.long)
    dataset = torch.utils.data.TensorDataset(torch_labe, torch_sentence1, torch_sentence2)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

# defile SNLI Model

In [0]:
class BaseLine(nn.Module):
    def __init__(self, config):
        super(BaseLine, self).__init__()

        self.embed = nn.Embedding(config["n_embed"], config["d_embed"])
        self.hbmp = HBMP(config)
        self.layer1 = nn.Linear(config["d_embed"], 32)
        self.layer2 = nn.Linear(config["d_embed"], 32)
        self.output = nn.Linear(32, config["n_output"])

    def forward(self, sentence1, sentence2):
#         sentence1_embed = self.embed(sentence1)
        sentence1_embed = self.hbmp(sentence1)
        sentence1_ctx = self.layer1(sentence1_embed)
        sentence1_ctx = sentence1_ctx.mean(1)

#         sentence2_embed = self.embed(sentence2)
        sentence2_embed = self.hbmp(sentence2)
        sentence2_ctx = self.layer2(sentence2_embed)
        sentence2_ctx = sentence2_ctx.mean(1)

        output = self.output(sentence1_ctx - sentence2_ctx)
        return output

In [0]:
# Hierarchical BiLSTM Max Pooling
class HBMP(nn.Module):
    def __init__(self, config):
        super(HBMP, self).__init__()
        
        self.config = config
        
        self.max_pool = nn.AdaptiveMaxPool1d(1)
        self.rnn1 = nn.LSTM(input_size=config["n_embed"],
                            hidden_size=config["d_embed"],
                            num_layers=config["n_layer"],
                            dropout=config["dropout"],
                            bidirectional=True)
        self.rnn2 = nn.LSTM(input_size=config["n_embed"],
                            hidden_size=config["d_embed"],
                            num_layers=config["n_layer"],
                            dropout=config["dropout"],
                            bidirectional=True)
        self.rnn3 = nn.LSTM(input_size=config["n_embed"],
                            hidden_size=config["d_embed"],
                            num_layers=config["n_layer"],
                            dropout=config["dropout"],
                            bidirectional=True)

    def forward(self, inputs):
        batch_size = inputs.size()[1]
        h_0 = c_0 = Variable(inputs.data.new(self.config["n_layer"],
                                             batch_size,
                                             self.config["d_embed"]).zero_())
        out1, (ht1, ct1) = self.rnn1(inputs, (h_0, c_0))
#         emb1 = self.max_pool(out1.permute(1,2,0)).permute(2,0,1)

#         out2, (ht2, ct2) = self.rnn2(inputs, (ht1, ct1))
#         emb2 = self.max_pool(out2.permute(1,2,0)).permute(2,0,1)

#         out3, (ht3, ct3) = self.rnn3(inputs, (ht2, ct2))
#         emb3 = self.max_pool(out3.permute(1,2,0)).permute(2,0,1)

#         emb = torch.cat([emb1, emb2, emb3], 2)
#         emb = emb.squeeze(0)

        return out

# Execute Training

In [0]:
vocab = load_vocab("gdrive/My Drive/data/snli_1.0/vocab.txt")
print("load_vocab .......................... %d" % len(vocab))
# train_labe, train_sentence1, train_sentence2 = load_data("gdrive/My Drive/data/snli_1.0/snli_1.0_train.txt", vocab)
train_labe, train_sentence1, train_sentence2 = load_data("gdrive/My Drive/data/snli_1.0/snli_1.0_test.txt", vocab) ## fast test only
print("load_data train ..................... %d" % len(train_labe))
dev_labe, dev_sentence1, dev_sentence2 = load_data("gdrive/My Drive/data/snli_1.0/snli_1.0_dev.txt", vocab)
print("load_data dev ....................... %d" % len(dev_labe))
test_labe, test_sentence1, test_sentence2 = load_data("gdrive/My Drive/data/snli_1.0/snli_1.0_test.txt", vocab)
print("load_data test ...................... %d" % len(test_labe))

config = {
          "n_embed": len(vocab), "d_embed": 32, "n_output": 3, "n_epoch": 10, "n_batch": 256,
          "n_layer": 1, "dropout": 0.1 ## HBMP
         }
snli = BaseLine(config=config)

seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(snli.parameters(), lr=0.01)


train_loader = build_tensor(train_labe, train_sentence1, train_sentence2, config["n_batch"])
dev_loader = build_tensor(dev_labe, dev_sentence1, dev_sentence2, config["n_batch"])
test_loader = build_tensor(test_labe, test_sentence1, test_sentence2, config["n_batch"])

epochs = []
dev_score = []
test_score = []

for epoch in range(config["n_epoch"]):
    epochs.append(epoch + 1)
    
    #
    # train
    #
    snli.train()
    train_loss = 0
    for i, data in enumerate(train_loader, 0):
        batch_label, batch_sentence1, batch_sentence2 = data

        optimizer.zero_grad()

        pred_label = snli(batch_sentence1, batch_sentence2)
        loss = loss_fn(pred_label, batch_label)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)
    
    snli.eval()
    #
    # dev evaluate
    #
    dev_match = np.zeros((len(dev_labe)))
    for i, data in enumerate(dev_loader, 0):
        batch_label, batch_sentence1, batch_sentence2 = data
        pred_label = snli(batch_sentence1, batch_sentence2)
        _, indices = pred_label.max(1)
        match = torch.eq(indices, batch_label).detach()
        dev_match[i * config["n_batch"]:(i+1) * config["n_batch"]] = match
    dev_accuracy = np.sum(dev_match) * 100 / len(dev_match)
    dev_score.append(dev_accuracy)

    #
    # test evaluate
    #
    test_match = np.zeros((len(test_labe)))
    for i, data in enumerate(test_loader, 0):
        batch_label, batch_sentence1, batch_sentence2 = data
        pred_label = snli(batch_sentence1, batch_sentence2)
        _, indices = pred_label.max(1)
        match = torch.eq(indices, batch_label).detach()
        test_match[i * config["n_batch"]:(i+1) * config["n_batch"]] = match
    test_accuracy = np.sum(test_match) * 100 / len(test_match)
    test_score.append(test_accuracy)
    
    print("[%d], loss: %.3f, dev: %.3f, test: %.3f" % (epoch + 1, train_loss, dev_accuracy, test_accuracy))


df = pd.DataFrame(data=np.array([dev_score, test_score]), columns=epochs, index=["dev", "test"])
display(df)

plt.plot(epochs, dev_score, label='dev')
plt.plot(epochs, test_score, label='test')
plt.show()

load_vocab .......................... 33827
load_data train ..................... 9824
load_data dev ....................... 9842
load_data test ...................... 9824


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


RuntimeError: ignored